In [ ]:
import json

import pandas as pd
import numpy as np
import networkx as nx

import torch
from torch_geometric.nn import GATv2Conv, global_max_pool, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import GraphSAINTNodeSampler
from matplotlib import pyplot as plt

from train_utils import *
from product_graph import *
from tqdm.notebook import tqdm
from torch_geometric.utils import to_dense_adj
from torch.nn import MSELoss
from product_graph import generate_parametric_product_graph
import networkx as nx
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.utils import from_scipy_sparse_matrix
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
dynamic_data = torch.tensor(np.log10(np.load("data/preprocessed/dynamic_data.npy", allow_pickle=True)), requires_grad=False)
S = torch.tensor(np.load("data/adjacency/sigma034.npy", allow_pickle=False), requires_grad=False)

data = create_forecasting_dataset(dynamic_data.T,
                                      splits = [0.8, 0.1, 0.1],
                                      pred_horizen=1,
                                      obs_window=4,
                                      verbose=0)
edge_index = torch.nonzero(S, as_tuple=False).t().contiguous()
edge_weight = S[edge_index[0], edge_index[1]]

criterion = MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

temporal_adj = np.array([[0, 0, 0, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

y_train_all = data['trn']['labels'].flatten()
PRICE_MEAN, PRICE_STD = np.mean(y_train_all), np.std(y_train_all)

In [ ]:
class GATv3Conv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, concat = True, heads=1) -> None:
        super().__init__()
        self.beta = torch.nn.Parameter(torch.tensor(0.5))
        self.conv = GATv2Conv(in_channels, out_channels, heads, concat, add_self_loops=False)

    def forward(self, x, edge_index, edge_weights):
        H, C = self.conv.heads, self.conv.out_channels

        if isinstance(x, torch.Tensor):
            assert x.dim() == 2
            x_l = self.conv.lin_l(x).view(-1, H, C)
            if self.conv.share_weights:
                x_r = x_l
            else:
                x_r = self.conv.lin_r(x).view(-1, H, C)
        else:
            raise TypeError("x must be a Tensor")

        assert x_l is not None
        assert x_r is not None

        # edge_updater_type: (x: PairTensor, edge_attr: OptTensor)
        alpha = self.conv.edge_updater(edge_index, x=(x_l, x_r), edge_attr=None)
        
        alpha = (1-self.beta) * alpha + self.beta * edge_weights.view(edge_weights.shape[0],1)
        # propagate_type: (x: PairTensor, alpha: Tensor)
        out = self.conv.propagate(edge_index, x=(x_l, x_r), alpha=alpha)

        if self.conv.concat:
            out = out.view(-1, self.conv.heads * self.conv.out_channels)
        else:
            out = out.mean(dim=1)

        if self.conv.bias is not None:
            out = out + self.conv.bias

        return out
    

class GATNN(torch.nn.Module):
    def __init__(self, in_dim=1, hidden_size=8, out_dim=1, in_head=8, out_head=8, linear_hidden=128, linear_out=1, pool_size=None, p=0.25) -> None:
        super().__init__()
        self.hid = hidden_size
        self.in_head = in_head
        self.out_head = out_head
        self.p = p
        self.linear_out = linear_out
        self.pool_size = pool_size
        
        self.conv1 = GATv3Conv(in_channels=in_dim, 
                               out_channels=self.hid, 
                               heads=self.in_head)
        
        # self.conv2 = GATv3Conv(in_channels=self.hid*in_head, 
        #                        out_channels=self.hid, 
        #                        heads=self.in_head)
        
        # self.conv3 = GATv3Conv(in_channels=self.hid*in_head, 
        #                        out_channels=self.hid, 
        #                        heads=self.in_head)
        
        self.conv4 = GATv3Conv(in_channels=self.hid*self.in_head, 
                               out_channels=out_dim, 
                               heads=self.out_head)
        
        self.linear1 = nn.Linear(out_dim*self.out_head, linear_hidden)
        self.linear2 = nn.Linear(linear_hidden, self.linear_out)

    def forward(self, x, edge_index, edge_weight, return_only_embedding=False):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.elu(x)
        # x = F.dropout(x, p=self.p, training=self.training)
        # x = self.conv2(x, edge_index, edge_weight)
        # x = F.elu(x)
        # # x = F.dropout(x, p=self.p, training=self.training)
        # x = self.conv3(x, edge_index, edge_weight)
        # x = F.elu(x)
        x = self.conv4(x, edge_index, edge_weight)
        x = F.elu(x)
        
        if (return_only_embedding):
            return x

        batch = torch.zeros(x.size(0), dtype=torch.long).cuda()
        x = global_max_pool(x, batch, size=x.shape[0]//4)

        # x = x.flatten()

        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        
        return x

In [ ]:
model = GATNN(in_dim=1, hidden_size=8, out_dim=8, in_head=8, out_head=8, linear_hidden=32, linear_out=1).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

model

In [ ]:
model = GATNN(in_dim=1, hidden_size=8, out_dim=8, in_head=8, out_head=8, linear_hidden=32, linear_out=1).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

model

In [ ]:
def converSignalToProductGraph(x):
    nrNodes = x.shape[0]
    new_x = torch.zeros(nrNodes*4)

    for i, row in enumerate(x):
        val1, val2, val3, val4 = row[0], row[1], row[2], row[3]
        new_x[i] = val1
        new_x[nrNodes + i] = val2
        new_x[nrNodes*2 + i] = val3
        new_x[nrNodes*3 + i] = val4

    return new_x.unsqueeze(dim=-1)

In [ ]:
# loop over the number of samples
train_samples = data['trn']['data'].shape[0]
val_samples = data['val']['data'].shape[0]
train_losses = []
val_losses = []

patience = 10
best_val_loss = float('inf')
counter = 0
flag = False

for epoch in tqdm(range(1, 21)):
    total_loss = 0
    for i in range(train_samples):
        # Create a torch geometric data over each graph 
        outer_batch = Data(x = torch.tensor(data['trn']['data'][i]), y = torch.tensor(data['trn']['labels'][i].squeeze()),
                        edge_index=edge_index, edge_weight = edge_weight) 
        
        train_loader = GraphSAINTNodeSampler(outer_batch, batch_size=200, num_steps=6)
        for inner_batch in train_loader:
            model.train()
            batch_loss = 0
            
            batch_adj = to_dense_adj(inner_batch.edge_index, edge_attr=inner_batch.edge_weight).squeeze(dim = 0)
            batch_adj = batch_adj.numpy()
            
            product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
            product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
            
            batch_x = converSignalToProductGraph(inner_batch.x).float().cuda()
            batch_x = (batch_x - PRICE_MEAN)
            
            optimizer.zero_grad()
            out = model(batch_x, product_edge_index.cuda(), product_edge_weight.float().cuda())
            
            batch_y = inner_batch.y.unsqueeze(dim=1).float().cuda()
            batch_y = (batch_y.float().cuda() - PRICE_MEAN)

            loss = criterion(out, batch_y)
            batch_loss += loss.item()
        
            loss.backward()
            optimizer.step()

        total_loss += batch_loss
        
    train_losses.append(total_loss/len(train_loader))
    val_loss = 0
    for i in range(val_samples):
        model.eval()
        # Create a torch geometric data over each graph 
        outer_batch = Data(x = torch.tensor(data['val']['data'][i]), y = torch.tensor(data['val']['labels'][i].squeeze()),
                        edge_index=edge_index, edge_weight = edge_weight) 
        
        val_loader = GraphSAINTNodeSampler(outer_batch, batch_size=200, num_steps=6)
        for val_batch in val_loader:
            with torch.no_grad():
                batch_loss = 0
                batch_adj = to_dense_adj(val_batch.edge_index, edge_attr=val_batch.edge_weight).squeeze(dim = 0)
                
                batch_adj = batch_adj.numpy()
                
                product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
                product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
                
                batch_x = converSignalToProductGraph(val_batch.x).cuda()
                batch_x = (batch_x - PRICE_MEAN)
                
                out = model(batch_x, product_edge_index.cuda(), product_edge_weight.float().cuda())

                batch_y = val_batch.y.unsqueeze(dim = 1).float().cuda()
                batch_y = (batch_y - PRICE_MEAN)

                batch_val_loss = criterion(out, batch_y)

                val_loss += batch_val_loss.item()
                
    val_losses.append(val_loss/len(val_loader))

    print(f'Epoch: {epoch}, Training Loss: {train_losses[-1]:.3f}, Validation Loss: {val_losses[-1]:.3f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        
    if counter >= patience:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'best_val_loss': best_val_loss,
            'train_losses': train_losses,
            'val_losses': val_losses,
        }, f'models/model_epoch_{epoch}.pt')
        flag = True
        break
    
if flag == False:
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'best_val_loss': best_val_loss,
        'train_losses': train_losses,
        'val_losses': val_losses,
    }, f'models/model_trained.pt')    

In [ ]:
model.conv1.beta, model.conv4.beta

In [ ]:
test_loss = 0
test_samples = data['tst']['data'].shape[0]
predicted_samples = []
true_samples = []
for i in range(test_samples):
    model.eval()
    # Create a torch geometric data over each graph 
    outer_batch = Data(x = torch.tensor(data['tst']['data'][i]), y = torch.tensor(data['tst']['labels'][i].squeeze()),
                    edge_index=edge_index, edge_weight = edge_weight) 
    
    test_loader = GraphSAINTNodeSampler(outer_batch, batch_size=200, num_steps=6)
    for test_batch in test_loader:
        with torch.no_grad():
            batch_loss = 0
            batch_adj = to_dense_adj(test_batch.edge_index, edge_attr=test_batch.edge_weight).squeeze(dim = 0)
            
            batch_adj = batch_adj.numpy()
            
            product_graph = generate_parametric_product_graph(s00 = 0, s01 = 1, s10 = 1, s11 = 1, A_T = temporal_adj, A_N = batch_adj, spatial_graph = None)
            product_edge_index, product_edge_weight = from_scipy_sparse_matrix(product_graph)
            
            batch_x = converSignalToProductGraph(test_batch.x).cuda()
            batch_x = (batch_x.float() - PRICE_MEAN)
            
            batch_y = test_batch.y.unsqueeze(dim = 1).float().cuda()
            batch_y = (batch_y.float().cuda() - PRICE_MEAN)

            out = model(batch_x, product_edge_index.cuda(), product_edge_weight.float().cuda())

            true_samples.extend(batch_y.detach().cpu())
            predicted_samples.extend(out.detach().cpu())

            batch_test_loss = criterion(out, batch_y)
            test_loss += batch_test_loss
                
print(f'Test Loss per Batch: {test_loss/(len(test_loader) * test_samples)}')

In [ ]:
y_pred_test = [10**(val.item() + PRICE_MEAN) for val in predicted_samples]
y_test = [10**(val.item()+PRICE_MEAN) for val in true_samples]

In [ ]:
print(f"R2: {r2_score(y_test, y_pred_test)}")

In [ ]:
print(f"The mean absolute error is: {mean_absolute_error(y_pred_test, y_test)}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.plot(np.arange(1.25, 4, 0.5), np.arange(1.25, 4, 0.5), color="Black", label="True Line", zorder=2)
ax.scatter(np.log10(y_test), np.log10(y_pred_test), edgecolors='black', color='dodgerblue', alpha=0.8, zorder=3)

ax.set_xlabel("True Price")
ax.set_ylabel("Predicted Price")
ax.set_title("Model Predictions Compared to Target Values (Log Scales)")

ax.grid(ls='--', zorder=0)

ax.legend()

fig.tight_layout()
fig.savefig("figures/model_predictions_small.pdf")